# Create a FABRIC Facility Port

This notebook shows how to use create an facility port to connect your FABRIC experiment to an external facility. 



## Import the FABlib Library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

## Create the Experiment Slice

The following creates a single node with basic compute capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

By default, the submit function will block until the node is ready and will display the progress of your slice being built.



In [2]:
slice_name = "Slice-AWSDX"

facility_port_site='FIU'

### Facility port info

#### Find the facility ports on the site
List the facility ports
- Facility Port Name
- Available VLAN range
- Currently allocated VLAN range

In [3]:
cell_output = 'pandas'
output_list = fablib.list_facility_ports(filter_function=lambda x: x['site_name'] == facility_port_site, output=cell_output)

Name,Site,Interface Name,VLAN Range,Allocated VLAN Range,Local Name,Device Name,Region
AmLight-EXP-Layer2-FIU,FIU,port+fiu-data-sw:HundredGigE0/0/0/23:facility+AmLight-EXP-Layer2-FIU-int,['4000-4019'],N/A,N/A,N/A,N/A
AmLight-Layer3-FIU,FIU,port+fiu-data-sw:HundredGigE0/0/0/23:facility+AmLight-Layer3-FIU-int,['3001-3008'],N/A,N/A,N/A,N/A


#### Select the Facility Port on the site

In [4]:
facility_port_name = 'AmLight-EXP-Layer2-FIU'

facility_port_vlan='4015'

### Create Slice

Create new_slice and add node(s) to the FABRIC topology

In [5]:
slice = fablib.new_slice(name=slice_name)

# Example: One VM on FABRIC-STAR with a NIC - sharedNIC (basic) or dedicated smartNIC (ConnectX_6)
node = slice.add_node(name=f"Node1", site='STAR')

node_iface = node.add_component(model='NIC_Basic', name="nic1").get_interfaces()[0]

Add facility port to the network service configuration along with the node interface(s)

In [6]:
facility_port = slice.add_facility_port(name=facility_port_name, site=facility_port_site, vlan=facility_port_vlan)
facility_port_interface =facility_port.get_interfaces()[0]

net = slice.add_l2network(name=f'net_facility_port', interfaces=[])
net.add_interface(node_iface)
net.add_interface(facility_port_interface)

In [7]:
print(f"facility_port.get_site(): {facility_port.get_site()}")
print(f"facility_port.get_fim_interface(): {facility_port.get_fim_interface()}")

facility_port.get_site(): FIU
facility_port.get_fim_interface(): {'node_id': '5f5e45b8-4a18-406f-84bb-57dbc05523df', 'name': 'AmLight-EXP-Layer2-FIU', 'site': 'FIU', 'type': 'Facility'}


Submit slice request to create

In [8]:
slice.submit();


Retry: 13, Time: 296 sec


ID,f42fb91f-e866-42c1-9630-846ff5fd0836
Name,Slice-AWSDX
Lease Expiration (UTC),2024-11-01 14:05:56 +0000
Lease Start (UTC),2024-10-31 14:05:56 +0000
Project ID,1ecd9d6a-7701-40fa-b78e-b2293c9526ed
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
ac569b3d-53c8-42a4-b378-31d4a42ecdc4,Node1,2,8,10,default_rocky_8,qcow2,star-w5.fabric-testbed.net,STAR,rocky,2001:400:a100:3030:f816:3eff:fef1:5de9,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3030:f816:3eff:fef1:5de9,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
50dc36c9-47da-48f5-ac77-b19c731418cd,net_facility_port,L2,L2STS,None,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-nic1-p1,p1,Node1,net_facility_port,100,config,,0A:2B:94:0B:E8:1A,eth1,eth1,fe80::ef0e:41f:9eca:fba1,4,HundredGigE0/0/0/1



Time to print interfaces 300 seconds


## Observe the Slice's Attributes


In [9]:
slice = fablib.get_slice(slice_name)

In [10]:
slice.show()
slice.list_nodes()
slice.list_networks()
slice.list_interfaces()


ID,f42fb91f-e866-42c1-9630-846ff5fd0836
Name,Slice-AWSDX
Lease Expiration (UTC),2024-11-01 14:05:56 +0000
Lease Start (UTC),2024-10-31 14:05:56 +0000
Project ID,1ecd9d6a-7701-40fa-b78e-b2293c9526ed
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
ac569b3d-53c8-42a4-b378-31d4a42ecdc4,Node1,2,8,10,default_rocky_8,qcow2,star-w5.fabric-testbed.net,STAR,rocky,2001:400:a100:3030:f816:3eff:fef1:5de9,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3030:f816:3eff:fef1:5de9,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
50dc36c9-47da-48f5-ac77-b19c731418cd,net_facility_port,L2,L2STS,None,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-nic1-p1,p1,Node1,net_facility_port,100,config,,0A:2B:94:0B:E8:1A,eth1,eth1,fe80::ef0e:41f:9eca:fba1,4,HundredGigE0/0/0/1


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-nic1-p1,p1,Node1,net_facility_port,100,config,,0A:2B:94:0B:E8:1A,eth1,eth1,fe80::ef0e:41f:9eca:fba1,4,HundredGigE0/0/0/1


## Run the Experiment

In [11]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Configure the interface(s) of the VM(s) with designated subnet
subnet = IPv4Network("192.168.1.0/24")
available_ips = list(subnet)[2:]

In [12]:
node1 = slice.get_node(name=f"Node1")        
node1_iface = node1.get_interface(network_name=f'net_facility_port') 
node1_addr = available_ips.pop(99)
print(f"node1_addr: {node1_addr}")
node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)

stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')

stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_physical_os_interface_name()} up')

stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_os_interface()} up')

node1_addr: 192.168.1.101
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 0a:2b:94:0b:e8:1a brd ff:ff:ff:ff:ff:ff
    altname enp7s0
    inet 192.168.1.101/24 scope global eth1
       valid_lft forever preferred_lft forever
    inet6 fe80::ef0e:41f:9eca:fba1/64 scope link noprefixroute 
       valid_lft forever preferred_lft forever


In [13]:
# Ping the node(s) that are active on the Facility Port side (e.g 192.168.1.10)
node1 = slice.get_node(name=f"Node1")     
node1_iface = node1.get_interface(network_name=f'net_facility_port') 

stdout, stderr = node1.execute(f'ping -c 5 192.168.1.10')   # This is expected to fail. 

PING 192.168.1.10 (192.168.1.10) 56(84) bytes of data.
From 192.168.1.101 icmp_seq=1 Destination Host Unreachable
From 192.168.1.101 icmp_seq=2 Destination Host Unreachable
From 192.168.1.101 icmp_seq=3 Destination Host Unreachable
From 192.168.1.101 icmp_seq=4 Destination Host Unreachable
From 192.168.1.101 icmp_seq=5 Destination Host Unreachable

--- 192.168.1.10 ping statistics ---
5 packets transmitted, 0 received, +5 errors, 100% packet loss, time 4115ms
pipe 3


# Adding SDXLIB to Network

In [14]:
import json
import pandas as pd
from pprint import pprint
import requests

from config import *
from sdxlib.sdx_client import SDXClient
from sdxlib.sdx_exception import SDXException

In [15]:
url = "http://67.17.206.205:8080/SDX-Controller"

## List All Available Ports


In [16]:
client=SDXClient(url)
client.get_available_ports()

Port ID,Status
urn:sdx:port:ampath.net:Ampath3:50,up
urn:sdx:port:ampath.net:Ampath2:50,up
urn:sdx:port:ampath.net:Ampath1:50,up
urn:sdx:port:sax.net:Sax01:50,up
urn:sdx:port:sax.net:Sax02:50,up
urn:sdx:port:tenet.ac.za:Tenet02:50,up
urn:sdx:port:tenet.ac.za:Tenet03:50,up
urn:sdx:port:tenet.ac.za:Tenet01:50,up


## Create Client

The below specification is the minimum information needed to create the layer 2 VPN. 

In [17]:
client_name = "Test SDX SC24 L2VPN"
client_endpoints = [
    {"port_id": "urn:sdx:port:ampath.net:Ampath2:50", "vlan": "301"}, 
    {"port_id": "urn:sdx:port:tenet.ac.za:Tenet02:50", "vlan": "301"},
]

In [18]:
client = SDXClient(url, client_name, client_endpoints)

In [19]:
try:
    response = client.create_l2vpn()  
    print("L2VPN creation successful!")
    pprint(response)
except SDXException as e:
    print(f"L2VPN creation failed: {e}") 

L2VPN creation successful!
{'service_id': 'f8cf59bf-fb54-445b-992f-c74d9123fafc'}


In [20]:
# Save the service_id for easy access later.
my_service_id = response['service_id']
my_service_id

'f8cf59bf-fb54-445b-992f-c74d9123fafc'

## List All Available L2VPNs

Both of the list methods have the option of query results being listed as a DataFrame(default) or as a JSON by passing the optional argument format='json'.

In [21]:
client.get_all_l2vpns()

,service_id,name,endpoints,description,notifications,scheduling,qos_metrics,ownership,creation_date,archived_date,status,state,counters_location,last_modified,current_path,oxp_service_ids
0,15e51695-bb9f-422e-8c6b-694296c3a2fb,new-connection,[{'port_id': 'urn:sdx:port:ampath.net:Ampath3:...,None,None,None,None,None,None,None,None,None,None,None,[{'port_id': 'urn:sdx:port:ampath.net:Ampath3:...,None
1,51de2931-7743-40f8-825e-97c08848baf1,new-connection,[{'port_id': 'urn:sdx:port:ampath.net:Ampath3:...,None,None,None,None,None,None,None,None,None,None,None,[{'port_id': 'urn:sdx:port:ampath.net:Ampath3:...,None
2,96191a70-d50a-4997-9112-c9b4765feca0,demo connection for SC video,[{'port_id': 'urn:sdx:port:ampath.net:Ampath1:...,This is a demo connection for SC video.,[{'email': 'sample@gmail.com'}],"{'end_time': '2024-11-07', 'start_time': '2024...","{'max_delay': {'strict': False, 'value': 45}, ...",None,None,None,None,None,None,None,[{'port_id': 'urn:sdx:port:ampath.net:Ampath1:...,None
3,eb930e37-e43f-4c99-bbd9-3524d1c8d557,new-connection,[{'port_id': 'urn:sdx:port:ampath.net:Ampath3:...,test description,None,None,None,None,None,None,None,None,None,None,[{'port_id': 'urn:sdx:port:ampath.net:Ampath3:...,None
4,f8cf59bf-fb54-445b-992f-c74d9123fafc,Test SDX SC24 L2VPN,[{'port_id': 'urn:sdx:port:ampath.net:Ampath2:...,None,None,None,None,None,None,None,None,None,None,None,[{'port_id': 'urn:sdx:port:ampath.net:Ampath2:...,None


## Query a Specific L2VPN

In [28]:
client.get_l2vpn(service_id=my_service_id)

,service_id,name,port_id,vlan,description,qos_metrics,notifications,ownership,creation_date,archived_date,status,state,counters_location,last_modified,current_path
0,f8cf59bf-fb54-445b-992f-c74d9123fafc,Test SDX SC24 L2VPN,urn:sdx:port:ampath.net:Ampath2:50,301,None,None,None,None,None,None,None,None,None,None,[{'port_id': 'urn:sdx:port:ampath.net:Ampath2:...
1,f8cf59bf-fb54-445b-992f-c74d9123fafc,Test SDX SC24 L2VPN,urn:sdx:port:tenet.ac.za:Tenet02:50,301,None,None,None,None,None,None,None,None,None,None,[{'port_id': 'urn:sdx:port:ampath.net:Ampath2:...


In [23]:
stdout, stderr = node1.execute(f'ping -c 5 192.168.1.10')   # This is expected to pass.

PING 192.168.1.10 (192.168.1.10) 56(84) bytes of data.
From 192.168.1.101 icmp_seq=1 Destination Host Unreachable
From 192.168.1.101 icmp_seq=2 Destination Host Unreachable
From 192.168.1.101 icmp_seq=3 Destination Host Unreachable
From 192.168.1.101 icmp_seq=4 Destination Host Unreachable
From 192.168.1.101 icmp_seq=5 Destination Host Unreachable

--- 192.168.1.10 ping statistics ---
5 packets transmitted, 0 received, +5 errors, 100% packet loss, time 4104ms
pipe 3


## Proceed With Reseach

**This is the area where network research will be completed.**

## Delete L2VPN

Please make certain to delete when you are finished. 

In [29]:
client.delete_l2vpn(service_id=my_service_id)

'OK'

## Delete the Slice

In [30]:
slice.delete()